In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from math import ceil
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

import sys
sys.path.append('..')
from utils.input_pipeline import get_image_folders
from utils.training import train, optimization_step
    
torch.cuda.is_available()

True

In [3]:
torch.backends.cudnn.benchmark = True

# Create data iterators

In [4]:
batch_size = 64

In [5]:
train_folder, val_folder = get_image_folders()

train_iterator = DataLoader(
    train_folder, batch_size=batch_size, num_workers=4,
    shuffle=True, pin_memory=True
)

val_iterator = DataLoader(
    val_folder, batch_size=256, num_workers=4,
    shuffle=False, pin_memory=True
)

# number of training samples
train_size = len(train_folder.imgs)
train_size

100000

In [6]:
# number of validation samples
val_size = len(val_folder.imgs)
val_size

10000

# Model

In [7]:
from densenet import DenseNet
model = DenseNet()
# load the model from step2
model.load_state_dict(torch.load('model_step2.pytorch_state'))

In [8]:
# create different parameter groups
weights = [
    p for n, p in model.named_parameters()
    if 'conv' in n or 'classifier.weight' in n
]
biases = [model.classifier.bias]
bn_weights = [
    p for n, p in model.named_parameters()
    if 'norm' in n and 'weight' in n
]
bn_biases = [
    p for n, p in model.named_parameters()
    if 'norm' in n and 'bias' in n
]

In [9]:
params = [
    {'params': weights, 'weight_decay': 1e-4},
    {'params': biases},
    {'params': bn_weights},
    {'params': bn_biases}
]
optimizer = optim.SGD(params, lr=1e-4, momentum=0.9, nesterov=True)
loss = nn.CrossEntropyLoss().cuda()
model = model.cuda()

# Train

In [10]:
n_epochs = 5
n_batches = ceil(train_size/batch_size)

# total number of batches in the train set
n_batches

1563

In [11]:
%%time
def optimization_step_fn(model, loss, x_batch, y_batch):
    return optimization_step(model, loss, x_batch, y_batch, optimizer)

all_losses = train(
    model, loss, optimization_step_fn,
    train_iterator, val_iterator, n_epochs
)
# epoch logloss  accuracy    top5_accuracy time  (first value: train, second value: val)

0  1.748 1.295  0.568 0.671  0.810 0.877  2096.644
1  1.631 1.224  0.592 0.689  0.827 0.886  2080.292
2  1.558 1.184  0.609 0.697  0.838 0.892  2084.515
3  1.493 1.154  0.622 0.704  0.848 0.893  2083.987
4  1.449 1.118  0.632 0.713  0.855 0.899  2083.928
CPU times: user 2h 38min 32s, sys: 15min 17s, total: 2h 53min 50s
Wall time: 2h 53min 49s


# Save

In [12]:
model.cpu();
torch.save(model.state_dict(), 'model_step3.pytorch_state')